<br><br><br><br><br><h1 style="font-size:2em;color:#2467C0">Predict survival on the Titanic and get familiar with ML basics</h1><br><br><br>

In [260]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer
import pyspark.sql.functions as F

In [261]:
# Load the data stored in CSV format as DataFrame.
trainData = spark.read.format("org.apache.spark.csv").option("header","true").option("inferSchema", "true").csv("data/train.csv")
testData = spark.read.format("org.apache.spark.csv").option("header","true").option("inferSchema", "true").csv("data/test.csv")
submissionData = spark.read.format("org.apache.spark.csv").option("header","true").option("inferSchema", "true").csv("data/gender_submission.csv")

In [262]:
trainData.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [263]:
# Create new Int field "SexInt" based on the existing String field "Sex"
# Sex "Male" = 1
# Sex "Female" = 0
newTrainData = trainData.withColumn('SexInt', (F.instr(trainData.Sex, 'female') == 0).cast('int'))
newTestData = testData.withColumn('SexInt', (F.instr(testData.Sex, 'female') == 0).cast('int'))
newTrainData.printSchema()
newTrainData.show(5)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexInt: integer (nullable = true)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|SexInt|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     1|
|         

In [264]:
newTrainData.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
PassengerId,891,446.0,257.3538420152301,1,891
Survived,891,0.3838383838383838,0.48659245426485753,0,1
Pclass,891,2.308641975308642,0.8360712409770491,1,3
Name,891,None,None,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""","van Melkebeke, Mr. Philemon"
Sex,891,None,None,female,male
Age,714,29.69911764705882,14.526497332334035,0.42,80.0
SibSp,891,0.5230078563411896,1.1027434322934315,0,8
Parch,891,0.38159371492704824,0.8060572211299488,0,6
Ticket,891,260318.54916792738,471609.26868834975,110152,WE/P 5735


In [265]:
featureColumns = ['Pclass', 'SexInt', 'Age', 'SibSp', 'Parch', 'Fare']

In [266]:
# Delete non useful fields
newTrainData = newTrainData.drop('Cabin', 'Ticket', 'PassengerId')
newTestData = newTestData.drop('Cabin', 'Ticket')
newTrainData = newTrainData.na.drop()
newTestData = newTestData.na.drop()
newTrainData.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexInt: integer (nullable = true)



In [267]:
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
assembledTrainData = assembler.transform(newTrainData)
assembledTestData = assembler.transform(newTestData)
assembledTrainData.show(5)
assembledTestData.show(5)

+--------+------+--------------------+------+----+-----+-----+-------+--------+------+--------------------+
|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexInt|            features|
+--------+------+--------------------+------+----+-----+-----+-------+--------+------+--------------------+
|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|   7.25|       S|     1|[3.0,1.0,22.0,1.0...|
|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|71.2833|       C|     0|[1.0,0.0,38.0,1.0...|
|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|  7.925|       S|     0|[3.0,0.0,26.0,0.0...|
|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|   53.1|       S|     0|[1.0,0.0,35.0,1.0...|
|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|   8.05|       S|     1|[3.0,1.0,35.0,0.0...|
+--------+------+--------------------+------+----+-----+-----+-------+--------+------+--------------------+
only showing top 5 rows

+--

In [268]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="Survived", outputCol="indexedLabel").fit(assembledTrainData)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(assembledTrainData)

In [269]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [270]:
#pipeline = Pipeline(stages=[dt])
#model = pipeline.fit(assembled)
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])
# Train model.  This also runs the indexers.
model = pipeline.fit(assembledTrainData)

In [271]:
# Make predictions.
predictions = model.transform(assembledTestData)

In [272]:
predictions.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexInt: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- indexedFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [273]:
extractedPredictions = predictions.select("PassengerId", "prediction")
output = submissionData.join(extractedPredictions,['PassengerId'],"inner")
predictionAndLabels = output.select("prediction", "Survived")

In [274]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictionAndLabels)
print("Test accuracy = %g " % (accuracy))

treeModel = model.stages[2]

Test accuracy = 0.945619 
